In [1]:
import morfeusz2
import pandas as pd

In [2]:
m = morfeusz2.Morfeusz()

In [3]:
articles = pd.read_csv('labeling/fresh_data.csv')\
    [['id', 'title', 'highlight', 'content', 'media_desc']]\

In [4]:
articles.head(1)

,id,title,highlight,content,media_desc
0,24162636,poznaniu powroci ul lutego wojewoda zbigniew h...,ostateczna decyzja nazwa ul lutego wrocic mape...,rok temu wojewoda wielkopolski zbigniew hoffma...,poznaniu najwieksze kontrowersje wzbudzila zmi...


In [5]:
articles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14412 entries, 0 to 14411
Data columns (total 5 columns):
id            14412 non-null int64
title         14411 non-null object
highlight     14411 non-null object
content       14412 non-null object
media_desc    12983 non-null object
dtypes: int64(1), object(4)
memory usage: 563.0+ KB


In [6]:
from unidecode import unidecode

In [7]:
def lemmatize_column(column):
    lemmatized_column = [] 
    for record in column.values:
        lemmatized_row = []
        try:
            for word in record.split(' '):
                lemmatized_row.append(unidecode(m.analyse(word)[0][2][1].split(':')[0]).lower())
            lemmatized_column.append(" ".join(lemmatized_row))
        except:
            lemmatized_column.append(None)
        
    return pd.Series(lemmatized_column, name = column.name)

In [8]:
lem_df_list = []
for column in [u'content', u'highlight', u'media_desc', u'title']:
    lem_df_list.append(lemmatize_column(articles[column]))
lem_df_list.append(articles['id'])

In [9]:
lem_df = pd.concat(lem_df_list, axis = 1)

In [10]:
lem_df.head(5)

,content,highlight,media_desc,title,id
0,rok ten wojewoda wielkopolski zbigniew hoffman...,ostateczny decyzja nazwa ul luty wrocic mape p...,poznanie najwieksze kontrowersja wzbudzila zmi...,poznanie powroci ul luty wojewoda zbigniew hof...,24162636
1,szef rad europejski donald tusk oglosil decyzj...,ue odsunac brexit maja izba gmin poprzec umowy...,brexitowy uber jadac brytyjczyk,merkel macrona chciec zebysmy przeszly histori...,24573799
2,wtorek gdansku wystapil cyrk zalewski udzialem...,mieszkancy osowa chca okno wystepowal cyrk zwi...,None,cyrk zwierzetami gdansku mieszkancy oburzony b...,24760624
3,wielki imienie rosja haslem kryc trwajaca pocz...,konca listopad rosjanin internetowy glosowac w...,szeremietiewo terminal glowne miedzynarodowe l...,chca kant kant rosja wybierac podniebny patronow,24203973
4,polityk partia rzadzacej mowia wprost mama czy...,seria pozarow skladowisk smieci calej polska m...,skawina dogaszac skladowiska smieci,krakowianka sklada skarge komisja europejski p...,23716514


In [11]:
from collections import Counter

In [12]:
lem_df = lem_df.fillna('')

In [13]:
bag_of_records = lem_df['title'] + ' ' + lem_df['highlight'] + ' ' + lem_df['content'] + ' ' + lem_df['media_desc']
bag_of_words = [x for x in [row for row in bag_of_records] for x in x.split(' ')]

In [14]:
idf_dict = {word:freq for word, freq in dict(Counter(bag_of_words)).items() if freq > 2}

In [15]:
from json import dump

In [16]:
with open('labeling/idf_dict', 'w') as f:
    dump(idf_dict, f)

In [17]:
lem_df.to_csv('labeling/lemmatized_articles.csv', index = False)